<center><b>EE 538 Spring 2021<br>
Low-Noise Analog Circuit Design<br>
University of Washington Electrical & Computer Engineering</b></center>


<b>Instructor: Jason Silver<br>
Assignment #4 (10 points)<br>
Due Sunday, May 2 (Submit on Canvas as a Jupyter Notebook)</b> 

*Please show your work*

In [1]:
# Imports
import os
import sys
import cmath
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import ltspice
import sympy as sp
from sympy.utilities.lambdify import lambdify
from scipy import signal
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib.ticker import LogLocator

In [2]:
def read_ltspice(file_name,ftype='trans',units='db'):
    cols = []
    arrs = []
    with open(file_name, 'r',encoding='utf-8') as data:
        for i,line in enumerate(data):
            if i==0:
                cols = line.split()
                arrs = [[] for _ in cols]
                continue
            parts = line.split()
            for j,part in enumerate(parts):
                arrs[j].append(part)
    df = pd.DataFrame(arrs,dtype='float64')
    df = df.T
    df.columns = cols
    if ftype=='trans':
        return df
    elif ftype=='ac':
        if units=='db':
            for col in cols:
                if df[col].str.contains(',').all():
                    df[f'Mag_{col}'] = df[col].apply(lambda x: x.split(',')[0])
                    df[f'Mag_{col}'] = df[f'Mag_{col}'].apply(lambda x: x[1:-2])
                    df[f'Mag_{col}'] = df[f'Mag_{col}'].astype('float64')
                    df[f'Phase_{col}'] = df[col].apply(lambda x: x.split(',')[1])
                    df[f'Phase_{col}'] = df[f'Phase_{col}'].apply(lambda x: x[0:-2])
                    df[f'Phase_{col}'] = df[f'Phase_{col}'].astype('float64')
        if units=='cartesian':
            for col in cols:
                if df[col].str.contains(',').all():
                    df[f'Re_{col}'] = df[col].apply(lambda x: x.split(',')[0])
                    df[f'Re_{col}'] = df[f'Re_{col}'].astype('float64')
                    df[f'Im_{col}'] = df[col].apply(lambda x: x.split(',')[1])
                    df[f'Im_{col}'] = df[f'Im_{col}'].astype('float64')
        df['Freq.'] = df['Freq.'].astype('float64')
        return df
    else:
        print('invalid ftype')

<b>Problem 1: Common-source JFET amplifier

<img src="img/CS_JFET.png" width=300 />

<br>
<center><b>Figure 1. Common-source amplifier </center>

The drain current of an $n$-channel JFET can be described as a function of $V_{gs}$ by

\begin{equation}
I_d = \beta(V_{gs} - V_{th})^2
\end{equation}

where $\beta = I_{DSS}/V_{th}^2$ (unrelated to the current gain parameter for BJTs). $I_{DSS}$ represents the maximum drain current that is reached when $V_{gs} = 0$. Using this expression, the transconductance can be expressed as

\begin{equation}
g_m = 2\cdot\sqrt{\beta I_D}
\end{equation}


For the following, use $\beta = 24 mA/V^2$ and $V_{gs} - V_{th} = 0.29V$

<u>*Analysis*</u>

__a)__ Assuming $I_{D} = 2mA$, $I_G = 2pA$, and $R_D = 4k\Omega$, determine noise density values for $i_n$ and $e_n$ of the common-source amplifier in Figure 1, ignoring $1/f$ noise.

__b)__ The $1/f$ drain current noise of a JFET can be expressed as

\begin{equation}
i_{nf}^2 = \dfrac{K_f\cdot I_D}{f}
\end{equation}

If $f_c$ is defined as the frequency at which the thermal and flicker noise densities are equal, determine $f_c$ if $K_f = 0.0021 fA$. 

__c)__ Assuming $R_s$ = $100k\Omega$, what are the signal gain and noise figure of the amplifier at $100Hz$?

<u>*Verification*</u>

__d)__ Verify your answers to a-c in Ltspice using the SPICE model of the LSK389 JFET transistor from Linear Systems. Set the DC value of $V_s$ to $-0.1V$. 

What happens to the noise figure if the temperature rises to $150C$? (this part only needs to be verified in SPICE, no calculation)